In [1]:
from datasets import load_dataset

poem = load_dataset("poem_sentiment")

Found cached dataset poem_sentiment (C:/Users/LENOVO/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
poem

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 105
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 104
    })
})

In [3]:
def label_int2str(row):
    return poem['train'].features['label'].int2str(row)
poem.set_format(type = "pandas")
df = poem['train'][:]
df['label_name'] = df['label'].apply(label_int2str)
df.head()

,id,verse_text,label,label_name
0,0,with pale blue berries. in these peaceful shad...,1,positive
1,1,"it flows so long as falls the rain,",2,no_impact
2,2,"and that is why, the lonesome day,",0,negative
3,3,"when i peruse the conquered fame of heroes, an...",3,mixed
4,4,of inward strife for truth and liberty.,3,mixed


In [4]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [5]:
def tokenize(batch):
    return tokenizer(batch['verse_text'],padding =True,truncation=True)

In [6]:
poem.reset_format()
poem_encoded = poem.map(tokenize,batched = True,batch_size = True)

Loading cached processed dataset at C:\Users\LENOVO\.cache\huggingface\datasets\poem_sentiment\default\1.0.0\4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099\cache-f5a2c3bcf6cb3658.arrow


Map:   0%|          | 0/105 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\LENOVO\.cache\huggingface\datasets\poem_sentiment\default\1.0.0\4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099\cache-53e95a9d39ca3d99.arrow


In [7]:
print(poem_encoded['train'][:5])

{'id': [0, 1, 2, 3, 4], 'verse_text': ['with pale blue berries. in these peaceful shades--', 'it flows so long as falls the rain,', 'and that is why, the lonesome day,', 'when i peruse the conquered fame of heroes, and the victories of mighty generals, i do not envy the generals,', 'of inward strife for truth and liberty.'], 'label': [1, 2, 0, 3, 3], 'input_ids': [[101, 2007, 5122, 2630, 22681, 1012, 1999, 2122, 9379, 13178, 1011, 1011, 102], [101, 2009, 6223, 2061, 2146, 2004, 4212, 1996, 4542, 1010, 102], [101, 1998, 2008, 2003, 2339, 1010, 1996, 10459, 14045, 2154, 1010, 102], [101, 2043, 1045, 7304, 3366, 1996, 11438, 4476, 1997, 7348, 1010, 1998, 1996, 9248, 1997, 10478, 11593, 1010, 1045, 2079, 2025, 21103, 1996, 11593, 1010, 102], [101, 1997, 20546, 27865, 2005, 3606, 1998, 7044, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [8]:
from transformers import AutoModelForSequenceClassification
import torch
num_labels = 4
device = "cuda" if torch.cuda.is_available() else "cpu"
id2label = {0: "negative", 1: "neutral", 2: "positive", 3: "mixed"}
label2id = {v: k for k, v in id2label.items()}
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels,id2label=id2label,
    label2id=label2id)
         .to(device))

D:\Jupyter\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\Jupyter\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [28]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(poem_encoded["train"])
model_name = f"{model_ckpt}-finetuned-rating-poem"
training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="steps",  # Đánh giá sau mỗi X bước
    eval_steps=50,  # Số bước mỗi lần đánh giá
    save_strategy="steps",  # Lưu sau mỗi X bước
     logging_steps=50,
    save_steps=50,  # Số bước mỗi lần lưu
    save_total_limit=1,  # Số lượng checkpoints tối đa để giữ lại
     adam_beta1=0.9,                  # beta1 cho thuật toán Adam
    adam_beta2=0.999,                # beta2 cho thuật toán Adam
    adam_epsilon=1e-8,                # epsilon cho thuật toán Adam
    load_best_model_at_end=True,  # Tải mô hình tốt nhất ở cuối quá trình
    metric_for_best_model="f1",  # Sử dụng F1 để đánh giá mô hình tốt nhất
    greater_is_better=True,  # Đặt là True nếu giá trị F1 càng cao càng tốt
    push_to_hub=False,  # Nếu bạn muốn đẩy mô hình của mình lên Hugging Face Hub
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=poem_encoded["train"],
                  eval_dataset = poem_encoded['validation'],
                  tokenizer=tokenizer)
trainer.train();

D:\Jupyter\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, verse_text. If id, verse_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 892
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 560
  Number of trainable parameters = 66,956,548


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.059900,1.024689,0.857143,0.861061
100,0.125700,1.123651,0.857143,0.849957
150,0.032000,1.134650,0.866667,0.856658
200,0.001200,1.218105,0.838095,0.837252
250,0.095400,1.042279,0.876190,0.866699
300,0.032300,1.056027,0.866667,0.871507
350,0.012800,1.115573,0.885714,0.880903
400,0.026900,1.170158,0.876190,0.868087
450,0.017200,1.196843,0.866667,0.867750
500,0.000400,1.190613,0.876190,0.876510


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, verse_text. If id, verse_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 8
Saving model checkpoint to distilbert-base-uncased-finetuned-rating-poem\checkpoint-50
Configuration saved in distilbert-base-uncased-finetuned-rating-poem\checkpoint-50\config.json
Model weights saved in distilbert-base-uncased-finetuned-rating-poem\checkpoint-50\pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-rating-poem\checkpoint-50\tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-rating-poem\checkpoint-50\special_tokens_map.json
Deleting older checkpoint [distilbert-base-uncased-finetuned-rating-poem\checkpoint-200] due to args.save_total_limit
The following 

Configuration saved in distilbert-base-uncased-finetuned-rating-poem\checkpoint-450\config.json
Model weights saved in distilbert-base-uncased-finetuned-rating-poem\checkpoint-450\pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-rating-poem\checkpoint-450\tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-rating-poem\checkpoint-450\special_tokens_map.json
Deleting older checkpoint [distilbert-base-uncased-finetuned-rating-poem\checkpoint-400] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, verse_text. If id, verse_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 8
Saving model checkpoint to distilbert-base-uncased-finetuned-rating-poem\checkpoint-500
Configura

In [30]:
preds_output = trainer.predict(poem_encoded["validation"])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, verse_text. If id, verse_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 105
  Batch size = 8


In [31]:
preds_output.metrics

{'test_loss': 1.1155728101730347,
 'test_accuracy': 0.8857142857142857,
 'test_f1': 0.8809033747809258,
 'test_runtime': 0.6743,
 'test_samples_per_second': 155.706,
 'test_steps_per_second': 20.761}

In [32]:
trainer.push_to_hub(commit_message="Training completed!")

Saving model checkpoint to distilbert-base-uncased-finetuned-rating-poem
Configuration saved in distilbert-base-uncased-finetuned-rating-poem\config.json
Model weights saved in distilbert-base-uncased-finetuned-rating-poem\pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-rating-poem\tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-rating-poem\special_tokens_map.json


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

'https://huggingface.co/VuaCoBac/distilbert-base-uncased-finetuned-rating-poem/tree/main/'